In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
orders_df = pd.read_csv('C:/Users/Admin/Downloads/orders.csv')
nodes_df = pd.read_csv('C:/Users/Admin/Downloads/nodes.csv')
test_df = pd.read_csv('C:/Users/Admin/Downloads/test.csv')

In [5]:
orders_df

,Id,running_time,completed_time,route_distance_km,delta_time
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0
1,-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0
2,-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0
3,5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0
4,1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0
...,...,...,...,...,...
4995,7096714159023973792,2022-01-24 21:10:38,2022-01-24 21:23:53,7.397,795.0
4996,-3836026425968071806,2022-01-24 15:10:27,2022-01-24 15:20:21,1.948,594.0
4997,2926216435675216636,2022-01-24 13:57:04,2022-01-24 14:03:18,2.547,374.0
4998,-6677307480063489707,2022-01-24 08:46:13,2022-01-24 08:55:34,3.013,561.0


In [6]:
orders_df['running_time'] =  pd.to_datetime(orders_df['running_time'], format='%Y-%m-%d %H:%M:%S')

In [7]:
orders_df['minutes'] =  orders_df['running_time'].dt.minute + orders_df['running_time'].dt.hour * 60

In [8]:
orders_df.sort_values('minutes', inplace=True)

In [9]:
nodes_count = nodes_df.groupby(['node_start','node_finish']).nunique().reset_index()
nodes_count = nodes_count.rename(columns={'Id': 'total_count'})
nodes_count = nodes_count[['node_start','node_finish','total_count' ]]
nodes_df = pd.merge(nodes_df, nodes_count, on = ['node_start','node_finish'], how='outer')


In [10]:
speed = []
for id in orders_df.Id.to_list():
    speed.append(nodes_df[nodes_df.Id == id].speed.mean())

In [11]:
total_nodes = []
for id in orders_df.Id.to_list():
    total_nodes.append(nodes_df[nodes_df.Id == id].total_count.sum())
orders_df['total_nodes'] = total_nodes

In [12]:
orders_df['speed'] = speed

In [13]:
orders_df = orders_df[orders_df.route_distance_km!=0.0]

In [14]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [15]:
import osmium
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.osm_data = []

    def node(self, n):
        tags = {tag.k: tag.v for tag in n.tags}
        self.osm_data.append({
            'id': n.id, 
            'lon': n.location.lon, 
            'lat': n.location.lat, 
            'tags': tags
        })

osmhandler = OSMHandler()
osmhandler.apply_file("C:/Users/Admin/Downloads/odessa_oblast.osm.pbf")

df = pd.DataFrame(osmhandler.osm_data)

df

,id,lon,lat,tags
0,10980417,30.738275,46.468524,{}
1,10980418,30.738516,46.469605,{}
2,10980419,30.736733,46.469758,{'highway': 'traffic_signals'}
3,10980421,30.730899,46.471836,{'highway': 'traffic_signals'}
4,10980422,30.731101,46.473256,{'highway': 'traffic_signals'}
...,...,...,...,...
4025490,10703831353,28.406402,45.521730,{}
4025491,10704537608,28.833946,45.358636,"{'access': 'permit', 'amenity': 'bicycle_parki..."
4025492,10704969778,30.654692,46.296579,"{'access': 'yes', 'amenity': 'parking', 'capac..."
4025493,10704969779,30.655059,46.296827,{}


In [16]:
orders_with_nodes = pd.merge(orders_df, nodes_df, on='Id', how='left')
unique_id = np.array(orders_with_nodes.Id.drop_duplicates())

first_d = {}
last_d = {}
d = {}
for i in unique_id:
    first = orders_with_nodes[orders_with_nodes.Id == i]['node_start']
    last = orders_with_nodes[orders_with_nodes.Id == i]['node_finish']
    first_d[i] = list(set(first)-set(last))
    last_d[i] = list(set(last)-set(first))
    if ( (len(first_d[i]) != 0) & (len(last_d[i]) != 0) ):
        first_d[i] = first_d[i][0]
        last_d[i] = last_d[i][0]

arbyz = []
arbyz2 = []
for key, value in first_d.items():
    arbyz.append( [key,value] )

for key, value in last_d.items():
    arbyz2.append([key,value] )


gluna = pd.DataFrame(arbyz,columns=['id','start'])
gluna2 = pd.DataFrame(arbyz2,columns=['id','finish'])
start_end = pd.merge(gluna,gluna2, on='id')

start_end

orders_df.rename(columns={'Id':'id'}, inplace=True)

start_end_merged = pd.merge(start_end, orders_df, on='id')
start_end_merged

C:\Users\Admin\AppData\Local\Temp\ipykernel_16936\4287719827.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df.rename(columns={'Id':'id'}, inplace=True)


,id,start,finish,running_time,completed_time,route_distance_km,delta_time,minutes,total_nodes,speed
0,1232212655921391683,965088602,1975639481,2022-01-24 00:30:04,2022-01-24 00:40:54,8.023,650.0,30,5642,43.358824
1,-8878794284622343010,2421901608,5216935096,2022-01-24 00:31:38,2022-01-24 00:43:48,6.361,730.0,31,7005,38.948529
2,5863100674479589673,4441087544,5039328705,2022-01-24 00:32:48,2022-01-24 00:44:11,6.781,683.0,32,3007,47.502703
3,-4358768120136565949,4732308202,8952394099,2022-01-24 00:32:33,2022-01-24 00:39:34,2.758,421.0,32,11673,36.888889
4,3710246373807088999,6878011672,7670771320,2022-01-24 00:33:17,2022-01-24 00:45:40,5.648,743.0,33,6681,44.476000
...,...,...,...,...,...,...,...,...,...,...
4994,-4669505154732059583,8952439672,290664089,2022-01-24 23:53:01,2022-01-25 00:04:48,6.109,707.0,1433,9659,37.929487
4995,1506157519405428084,4511419817,316676388,2022-01-24 23:53:59,2022-01-24 23:57:47,0.550,228.0,1433,493,24.526316
4996,2191822265040350571,4441087529,4627450042,2022-01-24 23:53:46,2022-01-25 00:04:58,3.374,672.0,1433,1662,23.072727
4997,-4036390670371710012,316676003,8952419257,2022-01-24 23:55:59,2022-01-25 00:05:48,4.853,589.0,1435,1572,32.387097


In [17]:
nodes_id_start = nodes_df.node_start.to_list()

lon_lat_start = df[df['id'].isin(nodes_id_start)]
df[df['id'].isin(nodes_id_start)]

lon_max = lon_lat_start['lon'].max()
lon_min = lon_lat_start['lon'].min()
lat_max = lon_lat_start['lat'].max()
lat_min = lon_lat_start['lat'].min()

In [18]:
def id_to_lon_lat_to_label(node_start_id):
  
      
  if(len(lon_lat_start[lon_lat_start['id']==node_start_id].lon.values) == 0):
    return '22'
  lon = lon_lat_start[lon_lat_start['id']==node_start_id].lon.values[0]
  lat = lon_lat_start[lon_lat_start['id']==node_start_id].lat.values[0]
  step1 = (lon_max-lon_min)/5
  x = (lon-lon_min)//step1
  step2 = (lat_max-lat_min)/8
  y = (lat-lat_min)//step2
  return str(int(x))+str(int(y))

In [28]:
start_cell = []
finish_cell = []
for i in start_end_merged.id:
    if isinstance(start_end_merged[start_end_merged.id == i]['start'].values[0], list):
        if(len(start_end_merged[start_end_merged.id == i]['start'].values[0]) == 0):
            start_cell.append('22')
        else:
            start_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].start.values[0][0]))
    else:
        start_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].start.values[0]))

    if isinstance(start_end_merged[start_end_merged.id == i]['finish'].values[0], list):
        if(len(start_end_merged[start_end_merged.id == i]['finish'].values[0]) == 0):
            finish_cell.append('22')
        else:
            finish_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].finish.values[0][0]))
    else:
        finish_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].finish.values[0]))
start_end_merged['start_cell'] = start_cell
start_end_merged['finish_cell'] = finish_cell

In [29]:
orders_df['start_cell'] = start_cell
orders_df['finish_cell'] = finish_cell

C:\Users\Admin\AppData\Local\Temp\ipykernel_16936\867508549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['start_cell'] = start_cell
C:\Users\Admin\AppData\Local\Temp\ipykernel_16936\867508549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['finish_cell'] = finish_cell


In [30]:
gg = orders_df.groupby(['start_cell','finish_cell']).count().reset_index()
gg = gg.rename(columns={'id': 'freq'})
gg = gg[['start_cell','finish_cell','freq' ]]
orders_df = pd.merge(orders_df, gg, on = ['start_cell','finish_cell'], how='outer')


In [31]:
cat_col = ['freq']
num_col = ['route_distance_km','minutes', 'total_nodes', 'speed']

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.15],
    'min_samples_split': [2, 4, 6]
}


column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore", sparse=False), cat_col),
    ('scaling', StandardScaler(), num_col)
])

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, eta =0.05, max_depth = 4)
pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression', xgb_model)
])

X = orders_df[['route_distance_km','minutes', 'total_nodes', 'freq', 'speed']]
X.freq = (X.freq>X.freq.mean())
y = orders_df['delta_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = pipeline.fit(X_train, y_train)

scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


C:\Users\Admin\AppData\Local\Temp\ipykernel_16936\1503739229.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.freq = (X.freq>X.freq.mean())


In [33]:
rmse

121.8537768193332

In [49]:
model = pipeline.fit(X, y)
y_pred = model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
rmse

114.99065682640975

In [43]:
def estimate(test_df, nodes_df):
    test_df['running_time'] =  pd.to_datetime(test_df['running_time'], format='%Y-%m-%d %H:%M:%S')
    test_df['minutes'] =  test_df['running_time'].dt.minute + test_df['running_time'].dt.hour * 60

    nodes_count = nodes_df.groupby(['node_start','node_finish']).nunique().reset_index()
    nodes_count = nodes_count.rename(columns={'Id': 'total_count'})
    nodes_count = nodes_count[['node_start','node_finish','total_count' ]]
    nodes_df = pd.merge(nodes_df, nodes_count, on = ['node_start','node_finish'], how='outer')

    total_nodes = []
    for id in test_df.Id.to_list():
        total_nodes.append(nodes_df[nodes_df.Id == id].total_count.sum())
    test_df['total_nodes'] = total_nodes

    speed = []
    for id in test_df.Id.to_list():
        speed.append(nodes_df[nodes_df.Id == id].speed.mean())
    test_df['speed'] = speed

    orders_with_nodes = pd.merge(test_df, nodes_df, on='Id', how='left')
    unique_id = np.array(orders_with_nodes.Id.drop_duplicates())

    first_d = {}
    last_d = {}
    d = {}
    for i in unique_id:
        first = orders_with_nodes[orders_with_nodes.Id == i]['node_start']
        last = orders_with_nodes[orders_with_nodes.Id == i]['node_finish']
        first_d[i] = list(set(first)-set(last))
        last_d[i] = list(set(last)-set(first))
        if ( (len(first_d[i]) != 0) & (len(last_d[i]) != 0) ):
            first_d[i] = first_d[i][0]
            last_d[i] = last_d[i][0]

    arbyz = []
    arbyz2 = []
    for key, value in first_d.items():
        arbyz.append( [key,value] )

    for key, value in last_d.items():
        arbyz2.append([key,value] )


    gluna = pd.DataFrame(arbyz,columns=['id','start'])
    gluna2 = pd.DataFrame(arbyz2,columns=['id','finish'])
    start_end = pd.merge(gluna,gluna2, on='id')

    start_end

    test_df.rename(columns={'Id':'id'}, inplace=True)

    start_end_merged = pd.merge(start_end, test_df, on='id')
    start_end_merged

    nodes_id_start = nodes_df.node_start.to_list()

    start_cell = []
    finish_cell = []
    for i in start_end_merged.id:
        if isinstance(start_end_merged[start_end_merged.id == i]['start'].values[0], list):
            if(len(start_end_merged[start_end_merged.id == i]['start'].values[0]) == 0):
                start_cell.append('22')
            else:
                start_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].start.values[0][0]))
        else:
            start_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].start.values[0]))

        if isinstance(start_end_merged[start_end_merged.id == i]['finish'].values[0], list):
            if(len(start_end_merged[start_end_merged.id == i]['finish'].values[0]) == 0):
                finish_cell.append('22')
            else:
                finish_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].finish.values[0][0]))
        else:
            finish_cell.append(id_to_lon_lat_to_label(start_end_merged[start_end_merged.id == i].finish.values[0]))

    test_df['start_cell'] = start_cell
    test_df['finish_cell'] = finish_cell
    gg = test_df.groupby(['start_cell','finish_cell']).count().reset_index()
    gg = gg.rename(columns={'id': 'freq'})
    gg = gg[['start_cell','finish_cell','freq' ]]
    test_df = pd.merge(test_df, gg, on = ['start_cell','finish_cell'], how='outer')

    X_test = test_df[['route_distance_km','minutes', 'total_nodes', 'freq', 'speed']]
    X_test.freq = (X_test.freq>X_test.freq.mean())
    return model.predict(X_test)
    

In [51]:
X_test_final = pd.read_csv('C:/Users/Admin/Downloads/final_test.csv')
X_nodes = pd.read_csv('C:/Users/Admin/Downloads/nodes_test.csv')

In [52]:
preds = estimate(X_test_final, X_nodes)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16936\902221496.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.freq = (X_test.freq>X_test.freq.mean())


In [53]:
preds.shape

(1000,)

In [59]:
result = X_test_final[['id']].copy()
result['Predicted'] = preds
result = result.rename(columns = {'id':'Id'})
result

,Id,Predicted
0,6198,522.330261
1,6417,664.465454
2,7054,839.590637
3,9628,822.946045
4,10283,690.501770
...,...,...
995,525706,857.532349
996,526604,781.615662
997,527213,777.455627
998,527520,638.939026


In [60]:
result.to_csv('predicts.csv', index=False)